In [92]:
import pandas as pd
import subprocess
import os

In [136]:
full_tab = pd.read_table('/vol/ek/share/peptide_docking_with_afold2_and_rosettAfold/AlphaFold_review/pep_length_and_ss2_data/full_replaced_tab_with_fastas_14092021.tsv',sep='\t')

In [137]:
full_tab

,f_id,final_id,final_chain,final_peptide,protein_fasta,peptide_fasta
0,1.1.5.13,3rc4,A,B,GSHMASMKKKGSVVIVGRINLSGDTAYAQQTRGEEGCQETSQTGRD...,PPPPPPPPSSTPC
1,1.1.5.147,2z9i,A,G,ANMPPGSVEQVAAKVVPSVVMLETDLGRQSEEGSGIILSAEGLILT...,GATV
2,1.1.5.153,1fiw,A,L,IIGGQDAAHGAWPWMVSLQIFTYHNNRRYHVCGGSLLNSQWLLTAA...,DNTTCDGPCGVRFRQNRQGGVR
3,1.1.5.198,2hwl,B,A,IVEGSDAEIGMSPWQVMLFRKSPQELLCGASLISDRWVLTAAHCLL...,TFGSGEADCGLRPLFEKKSLEDKTERELLESYIDGR
4,1.1.5.20,4fvd,A,C,GSGKFGQQSGAIYVGNFRVVNRHLATHNDWANLVWEDSSRDLLVSS...,GSITTLGKFG
...,...,...,...,...,...,...
122,877.1.1.15,1h6w,A,B,LSYPNATESVYGLTRYSTNDEAIAGVNNESSITPAKFTVALNNVFE...,SLNYIIKVKE
123,9.1.1.10,2qos,C,A,ASPISTIQPKANFDAQQFAGTWLLVAVGSAARFLQEQGHRAEATTL...,LRYDSTAERLY
124,9.2.1,6cct,A,B,GSGSKFRGHQKSKGNSYDVEVVLQHVDTGNSYLCGYLKIKGLTEEY...,PTLV
125,9.2.1.1,4z2o,A,P,FADDHAMSPDMKLLAGASNWVNQSGSVAQFVFTPSPTQPQTYEVSG...,SVATVSESLLTE


In [94]:
sifts_tab = pd.read_table('sifts_annotation_by_uniprot_chains.tsv',sep="\t", skiprows=1)
sifts_tab.columns = ['pdb', 'chain', 'sp_primary', 'res_beg', 'res_end', 'pdb_beg', 'pdb_end', 'sp_beg', 'sp_end']

In [95]:
ecod_tab = pd.read_csv('/vol/ek/share/databases/ecod/20210713/ecod.latest.domains.txt', skiprows=4, header=0, sep='\t')

In [96]:
# keep only ecod families that are within the af2 non redundant set, "full_tab"
ecod_tab_of_relevant_families = ecod_tab[ecod_tab.f_id.isin(full_tab.f_id.unique())]
# get all the pdbs of these families based on ecod
pdbs_of_families = ecod_tab_of_relevant_families.pdb.unique()

In [97]:
# keep only sifts entries for the pdbs matching ecod families from af2 non redundant
filtered_sifts = sifts_tab[sifts_tab.pdb.isin(pdbs_of_families)]

In [98]:
sum(filtered_sifts.pdb.isin(ecod_tab_of_relevant_families.pdb.unique())) == len(filtered_sifts)

True

In [63]:
# filt_sifts_grpby_pdb_and_chain = filtered_sifts.groupby(['pdb','chain']).size()
# multi_row_pdbs = filt_sifts_grpby_pdb_and_chain[filt_sifts_grpby_pdb_and_chain > 1]

In [99]:
# filter duplicate rows - any gapped (unresolved) segment in a pdb induces a new line in sifts 
filtered_no_dup = filtered_sifts.drop_duplicates(subset=['pdb','chain','sp_primary'])

In [100]:
# group the filtered sifts tab by pdb and chain, compute the size of each group
grp_sizs = filtered_no_dup.groupby(['pdb','chain']).size()
# keep only groups of size >=2, meaning more than 1 uniprot for a chain; 
# merge with the group series with the filtered sifts data
multi_uniprot_tab = grp_sizs[grp_sizs >= 2].reset_index().drop(0,axis=1).merge(filtered_no_dup, how='left',on=['pdb','chain'])

In [105]:
# # merge with the ecod data 
# # Note that NaNs could occure for cases where a chain doesn't have a mapped ecod family, e.g. a peptide chain. 
# multi_uniprot_with_families = multi_uniprot_tab.merge(ecod_tab_of_relevant_families[['f_id','pdb','chain']], how='left',on=['pdb','chain'])

In [ ]:
# check that indeed all families are also in the af2 non redun tab, after dropping NaNs
sum(multi_uniprot_with_families.dropna(subset=['f_id']).f_id.isin(full_tab.f_id.unique())) == len(multi_uniprot_with_families.dropna(subset=['f_id']))

In [171]:
multi_uniprot_matched_families = multi_uniprot_tab.merge(ecod_tab_of_relevant_families[['f_id','pdb','chain','unp_acc']].rename(columns={'unp_acc':'sp_primary'}))
multi_uniprot_matched_families.merge(full_tab, on=['f_id']).to_csv('multi_uniprot_matched_families.tsv',sep="\t", index=False)

In [182]:
multi_uniprot_tab

,pdb,chain,sp_primary,res_beg,res_end,pdb_beg,pdb_end,sp_beg,sp_end
0,1un5,A,P03950,2,38,None,37,25,61
1,1un5,A,P61823,39,43,38,42,64,68
2,1zvo,A,P01700,1,96,1,96,20,115
3,1zvo,A,P0CG04,109,214,109,214,1,106
4,1zvo,B,P01700,1,96,1,96,20,115
...,...,...,...,...,...,...,...,...,...
1287,7kxf,A,Q15788,272,285,None,528,683,696
1288,7l9p,X,Q86V20,2,16,2,None,5,19
1289,7l9p,X,Q6ZNX1,33,89,None,89,2,58
1290,7l9p,Y,Q86V20,2,16,None,None,5,19
